In [ ]:
###---(oﾟvﾟ)ノ---###
#Author Start
# hint Date: 2023-10-13 10:01:26
# hint LastEditors: Jupiter.Q.Peng
# hint LastEditTime: 2023-10-15 23:20:23
# hint Description: Version 1.0
# hint FilePath: \RotatE-SimTWD-Re-Rankd:\AxMyWorkBench\PythonProject\SmallTools\Rename_Tools\rename_tools.ipynb
# Author End
import os, re, numpy, logging
from rich.progress import track

def RT_ContainStr(str, target_str):
    pattern = r'\s+|[_-]'
    list_base = re.split(pattern, str.lower())
    list_target = re.split(pattern, target_str.lower())
    score = 0
    for i in list_target:
        if i in list_base:
            score += 1
    if score / len(list_base) > 7/9:
        return True
    else:
        return False

def RT_GetDirSize(dir):
    size = 0
    for root, dirs, files in os.walk(dir):
        size += sum([os.path.getsize(os.path.join(root, name)) for name in files])
    size = size / 1024 / 1024 / 1024
    return size

def RT_RenameFiles(path, rename_inject = "", controller = 0, logObject = None):
    '''
    :description: 
    :param path [*]
    :param rename_inject [*]
    :param contorller [int]
        0. 不重命名相似文件
        1: 重命名相似文件
    :return [*]
    '''
    if logObject != None:
        raise Exception("LogObject is None")
    else:
        # HINT 读取文件夹下的文件名列表
        filenameList = os.listdir(path)
        for filename in filenameList:
            # HINT 拼接文件路径，是当前的文件名
            file_path_current = os.path.join(path, filename)
            # HINT 判断是否是文件夹
            if os.path.isdir(file_path_current):
                # HINT 如果是文件夹，递归调用，将文件夹的名字作为参数传入
                RT_RenameFiles(file_path_current, filename, controller)
            else:
                # HINT 判断是否是初始的调用，若运行，此时的inject是本级文件夹名字
                if rename_inject != "":
                    # HINT 新的文件名(基础)
                    new_file_path = os.path.join(path, rename_inject)
                    # HINT 扩展名
                    if os.path.splitext(filename)[1] != "":
                        file_extension = os.path.splitext(filename)[1]
                    else:
                        file_extension = ".png"
                    # HINT 判断是否包含重命名注入的分数阈值
                    if RT_ContainStr(filename, rename_inject):
                        if controller == 0:
                            new_file_path = f"{file_path_current}"
                        else:
                            new_file_path = f"{new_file_path}_File_{filenameList.index(filename):04d}{file_extension}"
                    elif RT_ContainStr("_cover", filename) == True:
                        new_file_path = f"{new_file_path}_CoverS{file_extension}"
                    # HINT 若文件太几把长，直接重新命名
                    elif len(filename) > 80:
                        new_file_path = f"{new_file_path}_File_{filenameList.index(filename):04d}{file_extension}"
                    else:
                        new_file_path = f"{new_file_path}_{filename}"

                    # HINT 判断新的文件名是否存在，不存在就重命名
                    if controller != -1 and controller != None:
                        if os.path.exists(new_file_path) == False and file_path_current != new_file_path:
                            os.rename(file_path_current, new_file_path)
                            print(f"Rename: {file_path_current} -> {new_file_path}")
                        elif file_path_current == new_file_path:
                            print("New File is the Same Name")
                        else:
                            print("This File Is Exist")
                else:
                    print("Rename_Inject OR Filename_Contain_Rename_Inject")

def RT_AddFileSizeEnd(basepath, rule = "a", logObject = None):
    if logObject == None:
        raise Exception("LogObject is None")
    else:
        # HINT 基本变量
        path_list = next(os.walk(basepath))[1]
        dirSize = {}
        # HINT 获取文件夹大小
        for dir_name in path_list:
            dirSize[dir_name] = f"{RT_GetDirSize(os.path.join(basepath, dir_name)):.2f} GB"
        # HINT 重命名
        for dir_name in path_list:
            # HINT 原文件名
            sourceFileNamePath = os.path.join(basepath, dir_name)
            # HINT 修改规则
            if rule == "a":
                new_name = f"{dir_name}-[{dirSize[dir_name]}]"
            if rule == "m" or "M":
                # HINT Find Model Name
                pattenModelNameKeyWord = r"(@M.*?\[.*?\])"
                modelName = re.findall(pattenModelNameKeyWord, dir_name)
                if len(modelName) == 0:
                    print(f"Error Name: {dir_name}")
                    modelName = None
                elif len(modelName) == 1:
                    modelName = modelName[0]
                else:
                    lenList = []
                    [lenList.append(len(i)) for i in modelName]
                    lenList = numpy.array(lenList)
                    modelName = modelName[lenList.argmax()]
                # HINT Find Commit
                pattenCommitKeyWord = r"\](@.*)"
                commit = re.findall(pattenCommitKeyWord, dir_name)
                # HINT 控制重命名的操作
                if modelName != None:
                    new_name = f"{modelName}-[{dirSize[dir_name]}]{commit[0] if len(commit) != 0 else ''}"
                else:
                    new_name = None

            # HINT 控制重命名的操作
            if new_name != None:
                newFileNamePath = os.path.join(basepath, new_name)
                os.rename(sourceFileNamePath, newFileNamePath)
            if sourceFileNamePath != newFileNamePath:
                print(f"Rename: {dir_name} -> {new_name}")
            else:
                print(f"Same name {dir_name}")

# @ RUN
path_1 = r"E:\@S\@M"
base_path = path_1

RT_RenameFiles(base_path, "", controller = -1)
# RT_AddFileSizeEnd(base_path)


In [ ]:
import re, numpy

text = '''Rename: @M[Vita 21312lievna ] -> @M[Vitalievna]-[1.27 GB]
Rename: @M[Viva Fleur] -> @M[Viva Fleur]-[10.45 GB]@KKKKKKK@SSSSSSSSSS'''
pattenModelNameKeyWord = r"(@M.*?\[.*?\])"
pattenCommitKeyWord = r"\](@.*)"

modelName = re.findall(pattenCommitKeyWord, text)

lenList = []
[lenList.append(len(i)) for i in modelName]
lenList = numpy.array(lenList)
new_name = f"{modelName[lenList.argmax()]}"

commit_content = "".join(modelName)

print(modelName)
print(commit_content)



In [ ]:
import os, re, numpy, logging
from rich.progress import track

class RT:
    '''
    :description: 
    :return [*]
    '''
    
    def __init__(self, filePath, logObject = None):
        self.filePath = filePath
        
        if logObject == None:
            raise Exception("LogObject is None")
        else:
            self.logObject = logObject

    # HNIT 用于计算并判断是否包含相似字符串
    def RT_ContainStr(self, source_str:str, target_str:str):
        pattern = r'\s+|[_-]'
        list_base = re.split(pattern, source_str.lower())
        list_target = re.split(pattern, target_str.lower())
        score = 0
        for i in list_target:
            if i in list_base:
                score += 1
        if score / len(list_base) > 7/9:
            return True
        else:
            return False
    # 获取文件夹的大小
    def RT_GetDirSize(self, dir:str, sizeSetting:str = "GB"):
        for root, dirs, files in os.walk(dir):
            size += sum([os.path.getsize(os.path.join(root, name)) for name in files])
        if sizeSetting == "GB" or "gb":
            size = size / 1024 / 1024 / 1024
        elif sizeSetting == "MB" or "mb":
            size = size / 1024 / 1024
        else:
            raise Exception("SizeSetting Error Should Be GB or MB")
        return size
    # 重命名文件工具
    def RT_RenameFiles(self, path, rename_inject = "", controller = 0, logObject = None):
        '''
        :description: 
        :param path [*]
        :param rename_inject [*]
        :param contorller [int]
            0. 不重命名相似文件
            1: 重命名相似文件
        :return [*]
        '''
        if logObject != None:
            raise Exception("LogObject is None")
        else:
            # HINT 读取文件夹下的文件名列表
            filenameList = os.listdir(path)
            for filename in filenameList:
                # HINT 拼接文件路径，是当前的文件名
                file_path_current = os.path.join(path, filename)
                # HINT 判断是否是文件夹
                if os.path.isdir(file_path_current):
                    # HINT 如果是文件夹，递归调用，将文件夹的名字作为参数传入
                    self.RT_RenameFiles(file_path_current, filename, controller)
                else:
                    # HINT 判断是否是初始的调用，若运行，此时的inject是本级文件夹名字
                    if rename_inject != "":
                        # HINT 新的文件名(基础)
                        new_file_path = os.path.join(path, rename_inject)
                        # HINT 扩展名
                        if os.path.splitext(filename)[1] != "":
                            file_extension = os.path.splitext(filename)[1]
                        else:
                            file_extension = ".png"
                        # HINT 判断是否包含重命名注入的分数阈值
                        if self.RT_ContainStr(filename, rename_inject):
                            if controller == 0:
                                new_file_path = f"{file_path_current}"
                            else:
                                new_file_path = f"{new_file_path}_File_{filenameList.index(filename):04d}{file_extension}"
                        elif self.RT_ContainStr("_cover", filename) == True:
                            new_file_path = f"{new_file_path}_CoverS{file_extension}"
                        # HINT 若文件太几把长，直接重新命名
                        elif len(filename) > 80:
                            new_file_path = f"{new_file_path}_File_{filenameList.index(filename):04d}{file_extension}"
                        else:
                            new_file_path = f"{new_file_path}_{filename}"

                        # HINT 判断新的文件名是否存在，不存在就重命名
                        if controller != -1 and controller != None:
                            if os.path.exists(new_file_path) == False and file_path_current != new_file_path:
                                os.rename(file_path_current, new_file_path)
                                print(f"Rename: {file_path_current} -> {new_file_path}")
                            elif file_path_current == new_file_path:
                                print("New File is the Same Name")
                            else:
                                print("This File Is Exist")
                    else:
                        print("Rename_Inject OR Filename_Contain_Rename_Inject")
    # 重命名文件夹，增加大小
    def RT_AddFileSizeEnd(self, basepath, rule = "a", logObject = None):
        if logObject == None:
            raise Exception("LogObject is None")
        else:
            # HINT 基本变量
            path_list = next(os.walk(basepath))[1]
            dirSize = {}
            # HINT 获取文件夹大小
            for dir_name in path_list:
                dirSize[dir_name] = f"{self.RT_GetDirSize( os.path.join(basepath, dir_name)):.2f} GB"
            # HINT 重命名
            for dir_name in path_list:
                # HINT 原文件名
                sourceFileNamePath = os.path.join(basepath, dir_name)
                # HINT 修改规则
                if rule == "a":
                    new_name = f"{dir_name}-[{dirSize[dir_name]}]"
                if rule == "m" or "M":
                    # HINT Find Model Name
                    pattenModelNameKeyWord = r"(@M.*?\[.*?\])"
                    modelName = re.findall(pattenModelNameKeyWord, dir_name)
                    if len(modelName) == 0:
                        print(f"Error Name: {dir_name}")
                        modelName = None
                    elif len(modelName) == 1:
                        modelName = modelName[0]
                    else:
                        lenList = []
                        [lenList.append(len(i)) for i in modelName]
                        lenList = numpy.array(lenList)
                        modelName = modelName[lenList.argmax()]
                    # HINT Find Commit
                    pattenCommitKeyWord = r"\](@.*)"
                    commit = re.findall(pattenCommitKeyWord, dir_name)
                    # HINT 控制重命名的操作
                    if modelName != None:
                        new_name = f"{modelName}-[{dirSize[dir_name]}]{commit[0] if len(commit) != 0 else ''}"
                    else:
                        new_name = None

                # HINT 控制重命名的操作
                if new_name != None:
                    newFileNamePath = os.path.join(basepath, new_name)
                    os.rename(sourceFileNamePath, newFileNamePath)
                if sourceFileNamePath != newFileNamePath:
                    print(f"Rename: {dir_name} -> {new_name}")
                else:
                    print(f"Same name {dir_name}")

# @ RUN
path_1 = r"E:\@S\@M"
base_path = path_1

RT.RT_RenameFiles(base_path, "", controller = -1)
# RT_AddFileSizeEnd(base_path)